<a href="https://colab.research.google.com/github/firarru/Text-Mining/blob/main/Sistem%20rekomendasi%20game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [29]:
# pip install --upgrade pandas

In [30]:
df = pd.read_csv('/content/games_march2025_cleaned.csv', sep=',', on_bad_lines='skip', engine='python')
df

,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11941,655740,Tallowmere 2: Curse of the Kittens,2024-02-19,0,9.99,0,Lady Tallowmere's kittens have been acting str...,Lady Tallowmere's kittens have been acting str...,Lady Tallowmere's kittens have been acting str...,NaN,...,0,1922,0,0,1,"{'Action Roguelike': 220, '2D Platformer': 213...",80,297,-1,-1
11942,3196540,IdleTale,2024-09-20,0,0.00,0,Idletale Is An Incremental/clicker Rpg Game. Y...,Idletale Is An Incremental/clicker Rpg Game. Y...,IdleTale is an incremental clicker RPG game. K...,NaN,...,0,0,0,0,0,[],61,297,47,19
11943,600460,Dustwind,2018-08-15,0,24.99,0,STORY CAMPAIGN INCLUDED About the Game WELCOME...,"WELCOME TO DUSTWIND, A POST-APOCALYPTIC REAL-T...",ARE YOU READY FOR CLASSIC REALTIME TACTICS? Be...,“Dog-filled tactics skirmisher Dustwind has be...,...,0,0,0,0,0,"{'Post-apocalyptic': 215, 'Female Protagonist'...",74,297,-1,-1
11944,1586700,MARSUPILAMI - HOOBADVENTURE,2021-11-16,0,19.99,0,Wishlist Now About the Game A new dimension ha...,"A new dimension has been found in Palombia, we...",Discover the Marsupilamis as you have never se...,NaN,...,0,0,0,0,5,"{'Adventure': 192, 'Platformer': 188, '2D': 18...",94,297,92,14


In [31]:
df = df[['name', 'release_date', 'detailed_description', 'tags']]

In [32]:
df['release_year'] = pd.to_datetime(df['release_date']).dt.year

<ipython-input-32-4c1c43b0180d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = pd.to_datetime(df['release_date']).dt.year


In [33]:
df.isnull().sum()/100

,0
name,0.00
release_date,0.00
detailed_description,0.48
tags,0.00
release_year,0.00


In [34]:
df.dropna(inplace=True)
df.shape

<ipython-input-34-1e8442348ac0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


(11898, 5)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11898 entries, 0 to 11945
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  11898 non-null  object
 1   release_date          11898 non-null  object
 2   detailed_description  11898 non-null  object
 3   tags                  11898 non-null  object
 4   release_year          11898 non-null  int32 
dtypes: int32(1), object(4)
memory usage: 511.2+ KB


In [37]:
df = df[df['release_year'] >= 2010].reset_index(drop=True)

In [38]:
df['content'] = df['detailed_description'] + ' ' + df['tags']

In [40]:
df = df[['name', 'release_year', 'content']]

In [41]:
df

,name,release_year,content
0,Counter-Strike 2,2012,"For over two decades, Counter-Strike has offer..."
1,PUBG: BATTLEGROUNDS,2017,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ..."
2,Dota 2,2013,"The most-played game on Steam. Every day, mill..."
3,Grand Theft Auto V Legacy,2015,"When a young street hustler, a retired bank ro..."
4,Tom Clancy's Rainbow Six® Siege,2015,Edition Comparison Ultimate Edition The Tom Cl...
...,...,...,...
11839,Tallowmere 2: Curse of the Kittens,2024,Lady Tallowmere's kittens have been acting str...
11840,IdleTale,2024,Idletale Is An Incremental/clicker Rpg Game. Y...
11841,Dustwind,2018,STORY CAMPAIGN INCLUDED About the Game WELCOME...
11842,MARSUPILAMI - HOOBADVENTURE,2021,Wishlist Now About the Game A new dimension ha...
